In [3]:
import pandas as pd
import re
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup
from email import message_from_string


C:\Users\bechi\AppData\Local\Temp\ipykernel_19692\3697222624.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
data = pd.read_csv("C:/Users/bechi/Downloads/archive/emails.csv", nrows=70000)

In [5]:
data['message'][0]


"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden <Tim Belden/Enron@EnronXGate>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nHere is our forecast\n\n "

In [6]:

def clean_email(email_str):
    msg = message_from_string(email_str)
    
    from_address = msg.get('From')
    to_address = msg.get('To')
    subject = msg.get('Subject')
    date = msg.get('Date')
    body = ""   
    if msg.is_multipart():
        for part in msg.get_payload():
            if part.get_content_type() == 'text/html':
                html_content = part.get_payload()
                soup = BeautifulSoup(html_content, "html.parser")
                body = soup.get_text(separator=' ')
                break
    else:
        body = msg.get_payload()
    
    return from_address, to_address, subject, date, body

In [7]:
cleaned_data = pd.DataFrame([clean_email(data["message"][i]) for i in range(len(data))], 
                            columns=['from_address', 'to_address', 'subject', 'date', 'body'])

In [8]:
cleaned_data.isnull().sum()

from_address       0
to_address      3857
subject            0
date               0
body               0
dtype: int64

In [9]:
cleaned_data = cleaned_data.dropna()
cleaned_data=cleaned_data[cleaned_data["body"]!=""]

In [10]:
cleaned_data.head()

,from_address,to_address,subject,date,body
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast\n\n
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes the...
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy,\n\n Can you send me a schedule of the s..."
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.


In [11]:
duplicated_rows = cleaned_data.duplicated()
print(duplicated_rows.sum())
cleaned_data = cleaned_data.drop_duplicates()
len(cleaned_data)

24546


41597

In [12]:
  
def extract_email(email_str):
    left = ''
    right = ''
    email = '@'
    
    if '@' in email_str:
        left, right = email_str.split('@', 1)
        
        
        
        for i in right:
            if i.isalnum() or (i == '.' and right[0] != '.'):
                email += i
            else:
                break
                
        for i in left[::-1]:
            if i.isalnum() or (i == '.' and left[-1] != '.'):
                email = i + email
            else:
                break      
    
    return email


In [13]:
cleaned_data['from_address'] = cleaned_data['from_address'].apply(extract_email)
cleaned_data['to_address'] = cleaned_data['to_address'].apply(extract_email)

In [23]:
print(cleaned_data[(cleaned_data['from_address'] == '@') | (cleaned_data['to_address'] == '@')])
cleaned_data = cleaned_data[(cleaned_data['from_address'] != '@') & (cleaned_data['to_address'] != '@')]
cleaned_data = cleaned_data[~(cleaned_data['from_address'].str.startswith('.') | cleaned_data['from_address'].str.endswith('.'))]
cleaned_data=cleaned_data[~(cleaned_data['to_address'].str.startswith('.') | cleaned_data['to_address'].str.endswith('.'))]
print(len(cleaned_data))
cleaned_data["date"][0].split(" ")

Empty DataFrame
Columns: [from_address, to_address, subject, date, body]
Index: []
41432


['Mon,', '14', 'May', '2001', '16:39:00', '-0700', '(PDT)']

In [15]:
print(len(cleaned_data[cleaned_data['date'].str[3] == ',']))
cleaned_data = cleaned_data[cleaned_data['date'].str[3] == ',']
print(len(cleaned_data))

41591
41591


In [16]:
def months():
    month=[]
    a=[]
    count=0
    for date in cleaned_data['date']:
        day, subdate = date.split(',')
        if day=='':count += count
        subdate = subdate.split(' ')
        if(len(subdate)>7) |(len(subdate)<7):count += count
        if (len(subdate)==7): 
             if not subdate[2].isalpha():
               a.append(date)
             if(subdate[2]) not in month:
              month.append(subdate[2])
    print(month)  
    print(count)  
    return a

In [17]:
a=months()
print(a)

['May', 'Oct', 'Aug', 'Jul', 'Sep', 'Apr', 'Jun', 'Mar', 'Feb', 'Jan', 'Dec', 'Nov']
0
[]


In [18]:
prefixes = ['re:', 'RE:', 'rE:']

filtered_data = cleaned_data[cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(filtered_data))
print(len(cleaned_data[cleaned_data['subject'].str.startswith('Re:')]))


5596
7076


In [19]:
def clean_subject(subject):
    prefixes = ['re:', 'RE:', 'rE:']


    for prefix in prefixes:
        if subject.startswith(prefix):
            subject = subject.replace(prefix, 'Re:', 1)
            break
    prefixes = ['Fw:']

    for prefix in prefixes:
        if subject.startswith(prefix):
            subject = subject.replace(prefix, 'FW:', 1)
            break
    return subject


In [20]:
prefixes = ['fWD:',  'FwD:', 'fWd:', 'FWd:', 'FWD:', 'fwD:','fwd:','fW']
cleaned_data = cleaned_data[~cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(cleaned_data))

41586


In [21]:
cleaned_data['subject'] = cleaned_data['subject'].apply(clean_subject)
len(cleaned_data)

41586

In [24]:
cleaned_data.tail(10)

,from_address,to_address,subject,date,body
69985,ichen@haas.berkeley.edu,social@haas.berkeley.edu,Stanford University survey about terrorist att...,"Tue, 2 Oct 2001 21:30:07 -0700 (PDT)",Hi all\n\nThis is an interesting study being c...
69986,lisa.mellencamp@enron.com,michael.tribolet@enron.com,sce-urgent,"Tue, 2 Oct 2001 22:14:57 -0700 (PDT)","reliant guy, mike jines, called me about the d..."
69988,robert.frank@enron.com,jeff.dasovich@enron.com,FW: FERC refund investigation into Pacific Nor...,"Wed, 3 Oct 2001 03:50:24 -0700 (PDT)",Hey Jeff -see memo attached; it contains good ...
69990,bryan.gottfredson@enron.com,janine.migden@enron.com,URM Positions for OCT.,"Wed, 3 Oct 2001 07:26:56 -0700 (PDT)",Colleagues:\n\n\tUtility Risk management posit...
69991,michael.tribolet@enron.com,jeff.dasovich@enron.com,FW: Lay It On the Line Survey Results,"Wed, 3 Oct 2001 07:29:25 -0700 (PDT)",\n\n -----Original Message-----\nFrom: \tEnron...
69993,alan.comnes@enron.com,tom.alonso@enron.com,Mark to Market of DWR Contracts,"Wed, 3 Oct 2001 10:46:31 -0700 (PDT)",Tim asked me to forward this to West Desk trad...
69995,john.shelk@enron.com,j..kean@enron.com,New Congressional Report on California,"Wed, 3 Oct 2001 12:11:47 -0700 (PDT)",\n\nRay Alvarez brought to my attention today ...
69996,alan.comnes@enron.com,jeff.dasovich@enron.com,Re: Mark to Market of DWR Contracts,"Wed, 3 Oct 2001 13:07:21 -0700 (PDT)","Mo, please speak up if I got this wrong.\n\nTh..."
69997,d..steffes@enron.com,jeff.dasovich@enron.com,FW: SCE Settlement,"Wed, 3 Oct 2001 13:08:33 -0700 (PDT)",\nJeff --\n\nHow about a call tomorrow to disc...
69998,susan.mara@enron.com,d..steffes@enron.com,FW: CAISO Notice - Temporary Suspension of Pre...,"Wed, 5 Sep 2001 11:42:30 -0700 (PDT)",Here is the ultimate indignity. just talked t...
